# Import Libraries

In [1]:
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler, PowerTransformer, OrdinalEncoder
from sklearn.model_selection import train_test_split

# dagshub setup


In [2]:
%pip install mlflow dagshub

In [3]:
## Importing dagshub

import dagshub
dagshub.init(repo_owner='Ankitkumar1141', repo_name='Swiggy_delivery_time_prediction', mlflow=True)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

Output()



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=4e9fa397-1b4c-4b8e-9b29-c9b60559b318&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=3381ed916a94c5ca12fe196ba79827ebf2ef9d82559c734301b959ca82ea6bf2




Accessing as Ankitkumar1141

Initialized MLflow to track repo "Ankitkumar1141/Swiggy_delivery_time_prediction"

Repository Ankitkumar1141/Swiggy_delivery_time_prediction initialized!

In [4]:
## importing mlflow

import mlflow
mlflow.set_tracking_uri("https://dagshub.com/Ankitkumar1141/Swiggy_delivery_time_prediction.mlflow")

In [5]:
mlflow.set_experiment("Experiment 2(Model selection)")

<Experiment: artifact_location='mlflow-artifacts:/9f8914da6da84f7aac9c999bdd2585f4', creation_time=1770964481468, experiment_id='1', last_update_time=1770964481468, lifecycle_stage='active', name='Experiment 2(Model selection)', tags={}>

# Load the data

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
%cd /content/drive/MyDrive/mlproject

/content/drive/MyDrive/mlproject


In [8]:
df = pd.read_csv("preprocessed_data.csv")

In [9]:
import joblib
preprocessor = joblib.load("preprocessor.joblib")

In [10]:
df.isnull().sum()

,0
age,1854
ratings,1908
weather,525
traffic,510
vehicle_condition,0
type_of_order,0
type_of_vehicle,0
multiple_deliveries,993
festival,228
city,1198


# Impuatation

 **Split of Data**

In [11]:
X = df.drop("time_taken", axis=1)
y = df["time_taken"]

In [12]:
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size = 0.2,random_state = 42)

In [13]:
## Check for same columns in data and preprocessor

expected_cols = set(preprocessor.feature_names_in_)
incoming_cols = set(x_train.columns)

if expected_cols != incoming_cols:
    raise ValueError(
        f"Schema mismatch:\nMissing: {expected_cols - incoming_cols}\n"
        f"Extra: {incoming_cols - expected_cols}"
    )

 **Transforming target feature**

In [14]:
y_train_org = y_train.values.reshape(-1, 1)
y_test_org = y_test.values.reshape(-1, 1)

In [15]:
pt = PowerTransformer()

y_train = pt.fit_transform(y_train_org)
y_test = pt.transform(y_test_org)

**Transforming independent feature**

In [16]:
x_train = preprocessor.transform(x_train)
x_test = preprocessor.transform(x_test)

In [17]:
## Double check for missing values
np.isnan(x_train).sum()

np.int64(0)

# Model Training

In [18]:
%pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.9/413.9 kB 30.2 MB/s eta 0:00:00


In [19]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
import optuna

In [20]:
from sklearn.metrics import r2_score, mean_absolute_error

In [21]:
def objective(trial):
    with mlflow.start_run(nested=True):
        model_name = trial.suggest_categorical("model",["SVM","RF","KNN","GB","XGB","LGBM"])

        if model_name == "SVM":
            kernel_svm = trial.suggest_categorical("kernel_svm",["linear","poly","rbf"])
            if kernel_svm == "linear":
                c_linear = trial.suggest_float("c_linear",0,10)
                model = SVR(C=c_linear,kernel="linear")

            elif kernel_svm == "poly":
                c_poly = trial.suggest_float("c_poly",0,10)
                degree_poly = trial.suggest_int("degree_poly",1,5)
                model = SVR(C=c_poly,degree=degree_poly,
                            kernel="poly")

            else:
                c_rbf = trial.suggest_float("c_rbf",0,100)
                gamma_rbf = trial.suggest_float("gamma_rbf",0,10)
                model = SVR(C=c_rbf,gamma=gamma_rbf,
                            kernel="rbf")

        elif model_name == "RF":
            n_estimators_rf = trial.suggest_int("n_estimators_rf",10,200)
            max_depth_rf = trial.suggest_int("max_depth_rf",2,20)
            model = RandomForestRegressor(n_estimators=n_estimators_rf,
                                        max_depth=max_depth_rf,
                                        random_state=42,
                                        n_jobs=-1)

        elif model_name == "GB":
            n_estimators_gb = trial.suggest_int("n_estimators_gb",10,200)
            learning_rate_gb = trial.suggest_float("learning_rate_gb",0,1)
            max_depth_gb = trial.suggest_int("max_depth_gb",2,20)
            model = GradientBoostingRegressor(n_estimators=n_estimators_gb,
                                                learning_rate=learning_rate_gb,
                                                max_depth=max_depth_gb,
                                                random_state=42)

        elif model_name == "KNN":
            n_neighbors_knn = trial.suggest_int("n_neighbors_knn",1,25)
            weights_knn = trial.suggest_categorical("weights_knn",["uniform","distance"])
            model = KNeighborsRegressor(n_neighbors=n_neighbors_knn,
                                        weights=weights_knn,n_jobs=-1)

        elif model_name == "XGB":
            n_estimators_xgb = trial.suggest_int("n_estimators_xgb",10,200)
            learning_rate_xgb = trial.suggest_float("learning_rate_xgb",0.1,0.5)
            max_depth_xgb = trial.suggest_int("max_depth_xgb",2,20)
            model = XGBRegressor(n_estimators=n_estimators_xgb,
                                    learning_rate=learning_rate_xgb,
                                    max_depth=max_depth_xgb,
                                    random_state=42,
                                    n_jobs=-1)

        elif model_name == "LGBM":
            n_estimators_lgbm = trial.suggest_int("n_estimators_lgbm",10,200)
            learning_rate_lgbm = trial.suggest_float("learning_rate_lgbm",0.1,0.5)
            max_depth_lgbm = trial.suggest_int("max_depth_lgbm",2,20)
            model = LGBMRegressor(n_estimators=n_estimators_lgbm,
                                    learning_rate=learning_rate_lgbm,
                                    max_depth=max_depth_lgbm,
                                    random_state=42)


        # train the model
        model.fit(x_train,y_train.ravel())

        # log model params
        mlflow.log_params(model.get_params())

        # get the predictions
        y_pred_train = model.predict(x_train)
        y_pred_test = model.predict(x_test)

        # get the actual predictions values
        y_pred_train_org = pt.inverse_transform(y_pred_train.reshape(-1,1))
        y_pred_test_org = pt.inverse_transform(y_pred_test.reshape(-1,1))

        # calculate the error
        error = mean_absolute_error(y_test_org,y_pred_test_org)

        # log model_name
        mlflow.log_param("model",model_name)

        # log error
        mlflow.log_metric("MAE",error)

        return error

In [22]:
# create optuna study
study = optuna.create_study(direction="minimize",study_name="model_selection")

with mlflow.start_run(run_name="Best Model") as parent:
    # optimize the objective function
    study.optimize(objective,n_trials=30,n_jobs=-1)

    # log the best parameters
    mlflow.log_params(study.best_params)

    # log the best score
    mlflow.log_metric("best_score",study.best_value)

[I 2026-02-16 05:57:32,696] A new study created in memory with name: model_selection
[I 2026-02-16 06:02:51,070] Trial 1 finished with value: 4.826462998280723 and parameters: {'model': 'SVM', 'kernel_svm': 'poly', 'c_poly': 3.9357721226548015, 'degree_poly': 1}. Best is trial 1 with value: 4.826462998280723.


🏃 View run bright-flea-819 at: https://dagshub.com/Ankitkumar1141/Swiggy_delivery_time_prediction.mlflow/#/experiments/1/runs/a9129b6f438e4af39578203748ee94d8
🧪 View experiment at: https://dagshub.com/Ankitkumar1141/Swiggy_delivery_time_prediction.mlflow/#/experiments/1


[I 2026-02-16 06:02:54,632] Trial 2 finished with value: 3.382904529571533 and parameters: {'model': 'XGB', 'n_estimators_xgb': 107, 'learning_rate_xgb': 0.3196245617085808, 'max_depth_xgb': 8}. Best is trial 2 with value: 3.382904529571533.


🏃 View run wise-doe-71 at: https://dagshub.com/Ankitkumar1141/Swiggy_delivery_time_prediction.mlflow/#/experiments/1/runs/c4428f6c142749238ba49289b7e2351c
🧪 View experiment at: https://dagshub.com/Ankitkumar1141/Swiggy_delivery_time_prediction.mlflow/#/experiments/1


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
[I 2026-02-16 06:02:56,598] Trial 3 finished with value: 3.790575310895715 and parameters: {'model': 'LGBM', 'n_estimators_lgbm': 198, 'learning_rate_lgbm': 0.35869432934942813, 'max_depth_lgbm': 2}. Best is trial 2 with value: 3.382904529571533.


...

🏃 View run skillful-conch-311 at: https://dagshub.com/Ankitkumar1141/Swiggy_delivery_time_prediction.mlflow/#/experiments/1/runs/2984f933164046418db69494afa6a04f
🧪 View experiment at: https://dagshub.com/Ankitkumar1141/Swiggy_delivery_time_prediction.mlflow/#/experiments/1


[I 2026-02-16 06:53:45,103] Trial 29 finished with value: 4.28597597163064 and parameters: {'model': 'RF', 'n_estimators_rf': 102, 'max_depth_rf': 6}. Best is trial 23 with value: 3.253021259469535.


🏃 View run blushing-calf-719 at: https://dagshub.com/Ankitkumar1141/Swiggy_delivery_time_prediction.mlflow/#/experiments/1/runs/2b3660e5fdfe4be9850a694a81a4ee9e
🧪 View experiment at: https://dagshub.com/Ankitkumar1141/Swiggy_delivery_time_prediction.mlflow/#/experiments/1


[I 2026-02-16 07:02:41,659] Trial 9 finished with value: 5.926930126254036 and parameters: {'model': 'SVM', 'kernel_svm': 'rbf', 'c_rbf': 82.4621289782525, 'gamma_rbf': 2.344805063262826}. Best is trial 23 with value: 3.253021259469535.


🏃 View run suave-hound-273 at: https://dagshub.com/Ankitkumar1141/Swiggy_delivery_time_prediction.mlflow/#/experiments/1/runs/3afa5eb6f62143259189650fafe902fa
🧪 View experiment at: https://dagshub.com/Ankitkumar1141/Swiggy_delivery_time_prediction.mlflow/#/experiments/1
🏃 View run Best Model at: https://dagshub.com/Ankitkumar1141/Swiggy_delivery_time_prediction.mlflow/#/experiments/1/runs/99ed8100e81a427ca637e019b41af5d2
🧪 View experiment at: https://dagshub.com/Ankitkumar1141/Swiggy_delivery_time_prediction.mlflow/#/experiments/1


In [23]:
# best score

study.best_value

3.253021259469535

In [24]:
study.best_params

{'model': 'LGBM',
 'n_estimators_lgbm': 179,
 'learning_rate_lgbm': 0.12005934268541849,
 'max_depth_lgbm': 13}

In [25]:
lgbm_params = {
    "n_estimators": 179,
    "learning_rate": 0.12005934268541849,
    "max_depth": 13
}

In [26]:
lgbm = LGBMRegressor(**lgbm_params)

lgbm.fit(x_train,y_train.ravel())

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.018212 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 334
[LightGBM] [Info] Number of data points in the train set: 36401, number of used features: 28
[LightGBM] [Info] Start training from score -0.000000


LGBMRegressor(learning_rate=0.12005934268541849, max_depth=13, n_estimators=179)

In [27]:
# get the predictions
y_pred_train = lgbm.predict(x_train)
y_pred_test = lgbm.predict(x_test)

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


In [28]:
# get the predictions
y_pred_train = lgbm.predict(x_train)
y_pred_test = lgbm.predict(x_test)

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


In [29]:
y_pred_train_org = pt.inverse_transform(y_pred_train.reshape(-1,1))
y_pred_test_org = pt.inverse_transform(y_pred_test.reshape(-1,1))

In [33]:
from sklearn.metrics import mean_absolute_error, r2_score

print(f"The train error is {mean_absolute_error(y_train_org,y_pred_train_org):.2f} minutes")
print(f"The test error is {mean_absolute_error(y_test_org,y_pred_test_org):.2f} minutes")

The train error is 3.04 minutes
The test error is 3.25 minutes


In [35]:
print(f"The train r2 score is {r2_score(y_train_org,y_pred_train_org):.2f}")
print(f"The test r2 score is {r2_score(y_test_org,y_pred_test_org):.2f}")

The train r2 score is 0.83
The test r2 score is 0.81


In [36]:
# dataframe of results

study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_c_poly,params_c_rbf,params_degree_poly,params_gamma_rbf,params_kernel_svm,...,params_max_depth_rf,params_max_depth_xgb,params_model,params_n_estimators_gb,params_n_estimators_lgbm,params_n_estimators_rf,params_n_estimators_xgb,params_n_neighbors_knn,params_weights_knn,state
0,0,5.707550,2026-02-16 05:57:33.376014,2026-02-16 06:32:18.697574,0 days 00:34:45.321560,NaN,48.942941,NaN,2.030743,rbf,...,NaN,NaN,SVM,NaN,NaN,NaN,NaN,NaN,NaN,COMPLETE
1,1,4.826463,2026-02-16 05:57:33.377205,2026-02-16 06:02:51.067764,0 days 00:05:17.690559,3.935772,NaN,1.0,NaN,poly,...,NaN,NaN,SVM,NaN,NaN,NaN,NaN,NaN,NaN,COMPLETE
2,2,3.382905,2026-02-16 06:02:51.076773,2026-02-16 06:02:54.632089,0 days 00:00:03.555316,NaN,NaN,NaN,NaN,NaN,...,NaN,8.0,XGB,NaN,NaN,NaN,107.0,NaN,NaN,COMPLETE
3,3,3.790575,2026-02-16 06:02:54.636826,2026-02-16 06:02:56.598786,0 days 00:00:01.961960,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,LGBM,NaN,198.0,NaN,NaN,NaN,NaN,COMPLETE
4,4,3.259717,2026-02-16 06:02:56.601999,2026-02-16 06:03:37.089015,0 days 00:00:40.487016,NaN,NaN,NaN,NaN,NaN,...,15.0,NaN,RF,NaN,NaN,192.0,NaN,NaN,NaN,COMPLETE
5,5,3.283663,2026-02-16 06:03:37.092968,2026-02-16 06:03:38.926808,0 days 00:00:01.833840,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,LGBM,NaN,145.0,NaN,NaN,NaN,NaN,COMPLETE
6,6,3.364064,2026-02-16 06:03:38.929419,2026-02-16 06:03:41.784486,0 days 00:00:02.855067,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,LGBM,NaN,62.0,NaN,NaN,NaN,NaN,COMPLETE
7,7,5.705000,2026-02-16 06:03:41.792986,2026-02-16 06:47:57.289184,0 days 00:44:15.496198,NaN,86.086433,NaN,1.721345,rbf,...,NaN,NaN,SVM,NaN,NaN,NaN,NaN,NaN,NaN,COMPLETE
8,8,3.322151,2026-02-16 06:32:18.706993,2026-02-16 06:32:20.156948,0 days 00:00:01.449955,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,LGBM,NaN,48.0,NaN,NaN,NaN,NaN,COMPLETE
9,9,5.926930,2026-02-16 06:32:20.160048,2026-02-16 07:02:41.659509,0 days 00:30:21.499461,NaN,82.462129,NaN,2.344805,rbf,...,NaN,NaN,SVM,NaN,NaN,NaN,NaN,NaN,NaN,COMPLETE


In [38]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(lgbm,
                         x_train,
                         y_train,
                         scoring="neg_mean_absolute_error",
                         cv=5,n_jobs=-1)

scores

array([-0.36205326, -0.36085266, -0.35730471, -0.36201035, -0.35753129])

In [39]:
# optimization history plot

optuna.visualization.plot_optimization_history(study)

In [40]:
# partial coord plot

optuna.visualization.plot_parallel_coordinate(study,params=["model"])

In [41]:
# avg scores for all tested models

study.trials_dataframe().groupby("params_model")['value'].mean().sort_values()

,value
params_model,
LGBM,3.346890
RF,3.589575
XGB,3.615609
GB,3.925339
KNN,4.975162
SVM,5.541486
